In [5]:
# Zheen H. Suseyi
# 08/11/2023
# CS 340
# SNHU
# Setup the Jupyter version of Dash
#from jupyter_plotly_dash import JupyterDash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
import plotly.graph_objs as go
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
USER = "aacuser"
PASS = "admin"
host = "nv-desktop-services.approto.com"
port = 31522
database = "AAC"
collection = "animals"
shelter = AnimalShelter(USER, PASS, host, port, database, collection)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)
print(df)
## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)



#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)
image_filename = 'logocat.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Zheen Loves Programming!'))),
    
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
                        href='https://www.snhu.edu')), 
    
    # Setting up Pie Chart
    html.Div([
        html.Center(html.H4('Analysis Of Animal Breeds For Rescue Operations At Austin Shelters')),
        dcc.Graph(id="graph"),
        html.P("Pie Chart Filter Options: "),
        dcc.Dropdown(
            id='names',
                    options=[
                            {'label': 'Water Rescue', 'value': 'water_rescue'},
                            {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain_rescue'},
                            {'label': 'Disaster/Individual Tracking', 'value': 'disaster_rescue'},
                            {'label': 'All Animals', 'value': 'all animals'}
                            ],
                        value='mountain_rescue'
                        )
                      ]),

    # Setting up Radio Items 
    html.Div(className='buttonRow',
        style={'display': 'flex'},
            children=[
                dcc.RadioItems(
                    id='rescueOptions',
                        options=[
                            {'label': 'Water Rescue', 'value': 'water_rescue'},
                            {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain_rescue'},
                            {'label': 'Disaster/Individual Tracking', 'value': 'disaster_rescue'},
                            {'label': 'All Animals', 'value': 'all animals'}
                            ],
                        value='all animals'
                        )
                      ]),
    
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} 
            for i in df.columns
        ],
        # Enhancments for user readability
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    
    html.Br(),
    html.Hr(),
    
    # Setting up map
    html.Div(
        id='map-id',
        className='col s12 m6',
    ),
])

#############################################
# Interaction Between Components / Controller
#############################################

# Radio Items Callback
@app.callback(
    Output('datatable-id', 'data'), 
    [Input('rescueOptions', 'value')]
)

# Enhancing Dashboard with a Radio Items Filtering Option
def update_dashboard(value):
    
    # Query will change depending on what option the user picks
    if value == 'water_rescue':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        
    elif value == 'mountain_rescue':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        
    elif value == 'disaster_rescue':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
        
    elif value == 'all animals':
        query = {}
        
    else:
        return []
    
    df = pd.DataFrame.from_records(shelter.read(query))
    df.drop(columns=['_id'], inplace=True) 
    return df.to_dict('records')

# Callback for Radio Items
@app.callback(
        Output('datatable-id', 'style_data_conditional'),
        [Input('datatable-id', 'selected_columns')]
)

# Highlight when User Clicks
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback for Highlight
@app.callback(
    Output("graph", "figure"), 
    [Input("names", "value")] 
)

# Generating a Pie Chart with Filtering Options
def update_graphs(value):
    
    # Query will change depending on what option the user picks
    if value == 'water_rescue':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        
    elif value == 'mountain_rescue':
         query = {
             "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
             "sex_upon_outcome": "Intact Male",
             "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
         }
        
    elif value == 'disaster_rescue':
         query = {
             "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
             "sex_upon_outcome": "Intact Male",
             "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
         }
        
    elif value == 'all animals':
         query = {}
        
    else:
         return []
    
    df = pd.DataFrame.from_records(shelter.read(query))
    
    figure = px.pie(df, names='breed', title=f'Preferred Animals: {value}')
    return figure


# Callback for map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)

# Generating an interactive map
def update_map(viewData, index):
     dff = pd.DataFrame.from_dict(viewData)
 # Because we only allow single row selection, the list can 
 # be converted to a row index here
     if index is None:
       row = 0
     else: 
       row = index[0]

# Austin TX is at [30.75,-97.48]
     return [
             dl.Map(style={'width': '1000px', 'height': '500px'},
             center=[30.75,-97.48], zoom=10, children=[
             dl.TileLayer(id="base-layer-id"),
             # Marker with tool tip and popup
             # Column 13 and 14 define the grid-coordinates for 
             # the map
             # Column 4 defines the breed for the animal
             # Column 9 defines the name of the animal
             dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                children=[
                    dl.Tooltip(dff.iloc[row,4]),
                    dl.Popup([
                       html.H1("Animal Name"),
                       html.P(dff.iloc[row,9])
                   ])
                ])
             ])
          ]

app.run_server(debug=True)

       rec_num age_upon_outcome animal_id animal_type                   breed  \
0          1.0          3 years   A746874         Cat  Domestic Shorthair Mix   
1          4.0         7 months   A733653         Cat             Siamese Mix   
2         10.0         3 months   A664290         Cat  Domestic Shorthair Mix   
3         11.0           1 year   A721199         Dog  Dachshund Wirehair Mix   
4          2.0           1 year   A725717         Cat  Domestic Shorthair Mix   
...        ...              ...       ...         ...                     ...   
10018      NaN              NaN       NaN       mouse                     NaN   
10019      NaN              NaN       NaN    SushiCat                     NaN   
10020      NaN              NaN       NaN         NaN             chickenduck   
10021      NaN              NaN       NaN         NaN                     NaN   
10022      NaN              NaN       NaN         Dog                     NaN   

              color date_of